<a href="https://colab.research.google.com/github/KimJisanER/code_jjambbong/blob/main/3D_CNN_0719.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
with open(file='/content/drive/MyDrive/dataset_5_HTR/train/train_data.pickle', mode='rb') as f:
    train_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/train/train_3D_data.pickle', mode='rb') as f:
    train_3D_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/test/test_data.pickle', mode='rb') as f:
    test_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/test/test_3D_data.pickle', mode='rb') as f:
    test_3D_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/validation/val_data.pickle', mode='rb') as f:
    val_data = pickle.load(f)
with open(file='/content/drive/MyDrive/dataset_5_HTR/validation/val_3D_data.pickle', mode='rb') as f:
    val_3D_data = pickle.load(f)

In [3]:
train_data

,target_accession,chembl_id,Fingerprint,pChEMBL_value_Ki
7911,P28223,CHEMBL311614,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.16
11866,P28335,CHEMBL52592,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",8.00
6067,P41595,CHEMBL299031,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",5.90
14159,P50406,CHEMBL4075727,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.28
5208,P28221,CHEMBL519037,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",5.70
...,...,...,...,...
8251,P28223,CHEMBL3577344,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",5.44
4289,P08908,CHEMBL598561,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.62
509,P28222,CHEMBL370896,"[[1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0...",7.90
15539,P34969,CHEMBL3401491,"[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0...",7.05


In [5]:
def convert_data(data):
  data_array = np.array([dt for dt in data])
  return data_array

def convert_fpdata(data):
  fpdata_array = np.array([keras.utils.to_categorical(fp, num_classes=2) for fp in data])
  return fpdata_array

In [6]:
class CustomDataLoader(tf.keras.utils.Sequence):
    def __init__(self, dataframe, dictionary):
        self.dataframe = dataframe
        self.dictionary = dictionary
        self.indexes = dataframe.index.tolist()  # 데이터 프레임의 인덱스 사용

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, index):
        index_value = self.indexes[index]
        data_frame_row = self.dataframe['Fingerprint'].loc[index_value]
        label = self.dataframe['pChEMBL_value_Ki'].loc[index_value].astype(np.float32)
        dictionary_row = self.dictionary[index_value]

        data_frame_row = convert_data(data_frame_row)

        # 텐서로 변환 (예시: 텐서플로우의 from_tensor_slices 메서드 사용)
        data_frame_tensor = tf.convert_to_tensor(data_frame_row)
        dictionary_tensor = tf.convert_to_tensor(dictionary_row)
        labels_tensor = tf.convert_to_tensor(label)

        data_frame_tensor = tf.expand_dims(tf.convert_to_tensor(data_frame_row), axis=0)
        dictionary_tensor = tf.expand_dims(tf.convert_to_tensor(dictionary_row), axis=0)

        return  dictionary_tensor, data_frame_tensor, labels_tensor


In [7]:
# 이미지 입력 경로
image_input_shape = (128, 128, 128, 24)
image_input = keras.Input(shape=image_input_shape)
x1_5x5 = layers.Conv3D(64, (3, 3, 3), padding='same', activation='relu')(image_input)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Conv3D(64, (3, 3, 3), padding='same', activation='relu')(x1_5x5)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Conv3D(128, (3, 3, 3), padding='same', activation='relu')(x1_5x5)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Conv3D(256, (3, 3, 3), padding='same', activation='relu')(x1_5x5)

x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
x1_5x5 = layers.Flatten()(x1_5x5)

x1 = x1_5x5
x1 = layers.Dense(256, activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Dropout(0.2)(x1)
x1 = layers.Dense(128, activation='relu')(x1)
x1 = layers.BatchNormalization()(x1)

# 비트 입력 경로
bit_input_shape = shape=(2048, 2)
bit_input = keras.Input(shape=bit_input_shape)
x2 = layers.Flatten()(bit_input)

# 경로 병합
merged = layers.concatenate([x1, x2])
merged = layers.Dense(512, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)
merged = layers.Dropout(0.2)(merged)
merged = layers.Dense(256, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)
merged = layers.Dropout(0.2)(merged)
merged = layers.Dense(128, activation='relu')(merged)
merged = layers.BatchNormalization()(merged)


# 회귀 모델 출력 레이어 추가
outputs = layers.Dense(1)(merged)

# 모델 생성
model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

# 모델 컴파일
# model.compile(optimizer='adam', loss='mse')
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()


# 모델 요약
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 24)]                                                          
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  41536       ['input_1[0][0]']                
                                8, 64)                                                            
                                                                                                  
 max_pooling3d (MaxPooling3D)   (None, 64, 64, 64,   0           ['conv3d[0][0]']                 
                                64)                                                           

In [7]:
# # 이미지 입력 경로
# image_input_shape = (128, 128, 128, 24)
# image_input = keras.Input(shape=image_input_shape)
# x1_5x5 = layers.Conv3D(32, (5, 5, 5), padding='same', activation='relu')(image_input)

# x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
# x1_5x5 = layers.Conv3D(64, (5, 5, 5), padding='same', activation='relu')(x1_5x5)

# x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
# x1_5x5 = layers.Conv3D(64, (5, 5, 5), padding='same', activation='relu')(x1_5x5)

# x1_5x5 = layers.MaxPooling3D((2, 2, 2))(x1_5x5)
# x1_5x5 = layers.Flatten()(x1_5x5)

# x1 = x1_5x5
# x1 = layers.Dense(256, activation='relu')(x1)
# x1 = layers.BatchNormalization()(x1)
# x1 = layers.Dropout(0.2)(x1)
# x1 = layers.Dense(128, activation='relu')(x1)
# x1 = layers.BatchNormalization()(x1)

# # 비트 입력 경로
# bit_input_shape = shape=(2048, 2)
# bit_input = keras.Input(shape=bit_input_shape)
# x2 = layers.Flatten()(bit_input)

# # 경로 병합
# merged = layers.concatenate([x1, x2])
# merged = layers.Dense(512, activation='relu')(merged)
# merged = layers.BatchNormalization()(merged)
# merged = layers.Dropout(0.2)(merged)
# merged = layers.Dense(256, activation='relu')(merged)
# merged = layers.BatchNormalization()(merged)
# merged = layers.Dropout(0.2)(merged)
# merged = layers.Dense(128, activation='relu')(merged)
# merged = layers.BatchNormalization()(merged)


# # 회귀 모델 출력 레이어 추가
# outputs = layers.Dense(1)(merged)

# # 모델 생성
# model = keras.Model(inputs=[image_input, bit_input], outputs=outputs)

# # 모델 컴파일
# # model.compile(optimizer='adam', loss='mse')
# loss_fn = tf.keras.losses.MeanSquaredError()
# optimizer = tf.keras.optimizers.Adam()


# # 모델 요약
# model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 24)]                                                          
                                                                                                  
 conv3d_3 (Conv3D)              (None, 128, 128, 12  96032       ['input_3[0][0]']                
                                8, 32)                                                            
                                                                                                  
 max_pooling3d_3 (MaxPooling3D)  (None, 64, 64, 64,   0          ['conv3d_3[0][0]']               
                                32)                                                         

In [8]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tqdm import tqdm

# Mixed Precision Training 활성화
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

save_dir = "/content/drive/MyDrive/dataset_5_HTR/3D_model"

def train_one_epoch(model, data_loader, loss_fn, optimizer):
    total_loss = 0.0
    num_batches = 0

    for inputs1, inputs2, labels in tqdm(data_loader):
        with tf.GradientTape() as tape:
            # 모델의 순방향 전파
            logits = model([inputs1, inputs2])

            # 손실 계산
            loss_value = loss_fn(labels, logits)

        # 그래디언트 계산
        grads = tape.gradient(loss_value, model.trainable_variables)

        # 옵티마이저를 사용하여 모델 가중치 업데이트
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # 훈련 결과 출력
        total_loss += loss_value
        num_batches += 1
        # print(f'Loss:{loss_value.numpy()} experimental: {labels} predicted: {logits}' )

    average_loss = total_loss / num_batches
    return average_loss

def evaluate(model, data_loader, loss_fn):
    total_loss = 0.0
    num_batches = 0

    for inputs1, inputs2, labels in data_loader:
        logits = model([inputs1, inputs2])
        print(logits)
        loss_value = loss_fn(labels, logits)

        total_loss += loss_value.numpy()
        num_batches += 1

    average_loss = total_loss / num_batches
    return average_loss

def train(model, train_data_loader, valid_data_loader, loss_fn, optimizer, num_epochs):
    train_losses = []
    valid_losses = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # 훈련
        train_loss = train_one_epoch(model, train_data_loader, loss_fn, optimizer)
        train_losses.append(train_loss)
        print(f"Train Loss: {train_loss}")

        # 검증
        valid_loss = evaluate(model, valid_data_loader, loss_fn)
        valid_losses.append(valid_loss)
        print(f"Validation Loss: {valid_loss}")

        model.save(os.path.join(save_dir, f"model_epoch_{epoch + 1}_3x3_4layer.h5"))

    return train_losses, valid_losses


# 데이터 로더 인스턴스 생성
train_data_loader = CustomDataLoader(train_data.iloc[: , 2:], train_3D_data)
valid_data_loader = CustomDataLoader(val_data.iloc[: , 2:], val_3D_data)


# 모델 훈련
num_epochs = 10
train_losses, valid_losses = train(model, train_data_loader, valid_data_loader, loss_fn, optimizer, num_epochs)

Epoch 1/10


100%|██████████| 13268/13268 [52:02<00:00,  4.25it/s]


Train Loss: 1.3879289627075195
tf.Tensor([[7.683636]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.6202497]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.8593097]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.5331845]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.8700795]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.63685]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.8904643]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.6527205]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.5613365]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.8486123]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.775232]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.8497815]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.339828]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.5331845]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.584291]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.6226473]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.627099]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.7438593]], shape=(1, 1), dtype=float32)
tf.T

tf.Tensor([[6.7782874]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.798068]], shape=(1, 1), dtype=float32)
Validation Loss: 1.2083347684434569
Epoch 2/10


100%|██████████| 13268/13268 [51:41<00:00,  4.28it/s]


...

100%|██████████| 13268/13268 [51:50<00:00,  4.26it/s]


Train Loss: 1.007952094078064
tf.Tensor([[8.221345]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.303594]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.1473618]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.233006]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.90267]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.539453]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.3269663]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.6668196]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.583254]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.0537133]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.6569514]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.0219226]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.9400334]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.241944]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.4113975]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.561242]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.6704755]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.932674]], shape=(1, 1), dtype=float32)
tf.Tenso

tf.Tensor([[6.844882]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.762451]], shape=(1, 1), dtype=float32)
Validation Loss: 0.9796571743676915
Epoch 4/10


  2%|▏         | 236/13268 [00:55<51:27,  4.22it/s]


KeyboardInterrupt: ignored

In [ ]:
def evaluate(model, data_loader, loss_fn):
    total_loss = 0.0
    num_batches = 0

    for inputs1, inputs2 in data_loader:
        logits = model([inputs1, inputs2])
        print(logits)
        labels = inputs2[:, -1]
        loss_value = loss_fn(labels, logits)

        total_loss += loss_value.numpy()
        num_batches += 1

    average_loss = total_loss / num_batches
    return average_loss

In [ ]:
test_data_loader = CustomDataLoader(test_data.iloc[: , 2:], test_3D_data)

In [ ]:
evaluate(model, test_data_loader, loss_fn)

tf.Tensor([[6.8239546]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.9718256]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.7061443]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.137581]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.540615]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.996312]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.167155]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.953685]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.334463]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.7186604]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.7656183]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.9808693]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.509247]], shape=(1, 1), dtype=float32)
tf.Tensor([[8.152228]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.5831766]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.2787914]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.3950186]], shape=(1, 1), dtype=float32)
tf.Tensor([[7.837284]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.2832727]], shape=(1, 1)

0.9103905382871924

In [ ]:
model.predict()